In [4]:
%matplotlib inline

In [5]:
import os
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import widgets, Layout
from IPython.display import display
from nipype.interfaces.nipy import SpaceTimeRealigner, ComputeMask
from nipype.interfaces.fsl import BET, MeanImage, ApplyMask, SwapDimensions
from nipype.interfaces.afni import SkullStrip, Automask
import seaborn as sns
from nipype.interfaces.ants import N4BiasFieldCorrection, Registration
from pca_utils import pca_denoising

In [6]:
in_dir = '/home/julia/projects/lc/20181006_165517_JH_LC_rsfMRI_03_1_1_nifti/JHLCrsfMRI/'
out_dir = '/home/julia/projects/lc/JH_LC_rsfMRI_03_analysis/'
struct_file = in_dir +  'JHLCrsfMRI_20/JHLCrsfMRI_20.nii.gz'
struct_data = nb.load(struct_file).get_data()
struct_affine = nb.load(struct_file).affine
struct_header = nb.load(struct_file).header
atlas_file = '/home/julia/projects/allen_brain/P56_Atlas_different.nii'
#method_file = in_dir + 'JHLCrsfMRI_48/JHLCrsfMRI_48_method.npy'
#par_file = in_dir +'JHLCrsfMRI_48/JHLCrsfMRI_48_visu_pars.npy'
func_file =  in_dir + 'JHLCrsfMRI_23/JHLCrsfMRI_23.nii.gz'
func_data = nb.load(func_file).get_data()
func_affine = nb.load(func_file).affine
func_header = nb.load(func_file).header
#method = np.load(method_file)[()]
#pars = np.load(par_file)[()]
#TR = pars['VisuAcqRepetitionTime']

In [4]:
if not os.path.isdir(out_dir+'struct/single_vols'):
    if not os.path.isdir(out_dir+'struct/'):
        if not os.path.isdir(out_dir):
            os.mkdir(out_dir)
        os.mkdir(out_dir+'struct/')
    os.mkdir(out_dir+'struct/single_vols')

### Structural data: denoising, bias field correction, average, mask

In [5]:
mp_struct_data, mp_struct_sigmas, mp_struct_components = pca_denoising(struct_data)
nb.save(nb.Nifti1Image(mp_struct_data, struct_affine, struct_header), out_dir+'struct/struct_mp_data.nii.gz')
nb.save(nb.Nifti1Image(mp_struct_sigmas, struct_affine, struct_header), out_dir+'struct/struct_mp_sigmas.nii.gz')
nb.save(nb.Nifti1Image(mp_struct_components, struct_affine, struct_header), out_dir+'struct/struct_mp_components.nii.gz')

In [ ]:
fig=plt.figure(figsize=(15,5))
sns.kdeplot(struct_data.flatten(), label='orig')
sns.kdeplot(mp_struct_data.flatten(), label='denoised')
plt.legend()
sns.despine()

In [7]:
for i in range(mp_struct_data.shape[3]):
    nb.save(nb.Nifti1Image(mp_struct_data[:,:,:,i], struct_affine, struct_header),
            out_dir + 'struct/single_vols/struct_vol%s.nii.gz'%i)
    n4 = N4BiasFieldCorrection(input_image=out_dir + 'struct/single_vols/struct_vol%s.nii.gz'%i, dimension=3,
                               n_iterations=[100,100,100,100], convergence_threshold=0.0,
                               output_image= out_dir + 'struct/single_vols/struct_vol%s_corrected.nii.gz'%i)
    n4.run()

In [8]:
corrected_data = []
for i in range(mp_struct_data.shape[3]):
    corrected_data.append(nb.load(out_dir + 'struct/single_vols/struct_vol%s_corrected.nii.gz'%i).get_data())

nb.save(nb.Nifti1Image(np.average(corrected_data[:], axis=0), struct_affine, struct_header),
        out_dir + 'struct/struct_avg_corrected.nii.gz')

In [23]:
skull = SkullStrip(in_file=out_dir + 'struct/single_vols/struct_vol0_corrected.nii.gz', 
                   out_file=out_dir + 'struct/struct_mask.nii.gz', outputtype='NIFTI_GZ',
                   args='-rat -push_to_edge')

skull.run()

181011-14:31:17,726 interface INFO:
	 stderr 2018-10-11T14:31:17.726355:*+ WARNING:   If you are performing spatial transformations on an oblique dset, 
181011-14:31:17,728 interface INFO:
	 stderr 2018-10-11T14:31:17.726355:  such as /home/julia/projects/lc/JH_LC_rsfMRI_03_analysis/struct/single_vols/struct_vol0_corrected.nii.gz,
181011-14:31:17,731 interface INFO:
	 stderr 2018-10-11T14:31:17.726355:  or viewing/combining it with volumes of differing obliquity,
181011-14:31:17,734 interface INFO:
	 stderr 2018-10-11T14:31:17.726355:  you should consider running: 
181011-14:31:17,737 interface INFO:
	 stderr 2018-10-11T14:31:17.726355:     3dWarp -deoblique 
181011-14:31:17,737 interface INFO:
	 stderr 2018-10-11T14:31:17.726355:  on this and  other oblique datasets in the same session.
181011-14:31:17,741 interface INFO:
	 stderr 2018-10-11T14:31:17.726355: See 3dWarp -help for details.
181011-14:31:17,743 interface INFO:
	 stderr 2018-10-11T14:31:17.726355:++ Oblique dataset:/home/j

In [24]:
mask = nb.load(out_dir + 'struct/struct_mask.nii.gz').get_data()
binmask = np.where(mask>0, 1, 0)

struct_masked = nb.load(out_dir + 'struct/struct_avg_corrected.nii.gz').get_data() * binmask

In [29]:
nb.save(nb.Nifti1Image(binmask, struct_affine, struct_header),
        out_dir + 'struct/struct_mask.nii.gz')

nb.save(nb.Nifti1Image(struct_masked, struct_affine, struct_header),
        out_dir + 'struct/struct_masked.nii.gz')

In [48]:
weighted_data = []
for d in range(len(corrected_data)):
    weighted_data.append(corrected_data[d]*np.square(d+1))
nb.save(nb.Nifti1Image(np.average(weighted_data[:], axis=0)*binmask, struct_affine, struct_header),
    out_dir + 'struct/struct_weighted_avg_masked.nii.gz')

In [57]:
reg_struct = Registration(fixed_image = atlas_file,
                          moving_image = out_dir + 'struct/struct_masked.nii.gz',
                          output_warped_image = out_dir + 'struct/struct2atlas_lin.nii.gz',
                          output_transform_prefix = out_dir + 'struct/struct2atlas_lin_',
                          dimension = 3,
                          transforms = ['Rigid'], 
                          metric = ['GC'], 
                          transform_parameters = [(0.1,)],
                          metric_weight = [1],
                          radius_or_number_of_bins = [64],
                          sampling_percentage = [0.2],
                          sampling_strategy = ['Regular'],
                          convergence_threshold = [1.e-16],
                          convergence_window_size = [30],
                          smoothing_sigmas = [[1, 0]], # reduces this a lot compared to SAMRI (4,2,1)
                          sigma_units = ['vox'],
                          shrink_factors = [[2, 1]],
                          use_estimate_learning_rate_once = [False], # if the fixed_image is not acquired similarly to the moving_image (e.g. RARE to histological (e.g. AMBMC)) this should be False
                          use_histogram_matching = [True],
                          number_of_iterations = [[3000, 2000]],
                          write_composite_transform = True,
                          collapse_output_transforms = True,
                          winsorize_lower_quantile = 0.005,
                          winsorize_upper_quantile = 0.995,
                          args = '--float',
                          num_threads = 3,
                          initial_moving_transform_com = True,
                         )
reg_struct.run()

In [56]:
reg_struct = Registration(fixed_image = atlas_file,
                          moving_image = out_dir + 'struct/struct_masked.nii.gz',
                          # in contrast to SAMRI included rigid step, CC for nonlinear, changed transform_parameters
                          dimension = 3,
                          transforms = ['Rigid','Affine', 'SyN'], 
                          metric = ['GC', 'MI','MI'], 
                          transform_parameters = [(0.1,), (0.1,), (0.1, 2.0, 0.2)],
                          metric_weight = [1, 1, 1],
                          radius_or_number_of_bins = [64, 16, 16],
                          sampling_percentage = [0.2, 0.3, 0.3],
                          sampling_strategy = ['Regular', None, None],
                          convergence_threshold = [1.e-16, 1.e-32, 1.e-8],
                          convergence_window_size = [30, 30, 30],
                          smoothing_sigmas = [[1, 0], [1, 0], [1, 0]], # reduces this a lot compared to SAMRI (4,2,1)
                          sigma_units = ['vox', 'vox', 'vox'],
                          shrink_factors = [[2, 1],[1, 1], [1, 1]],
                          use_estimate_learning_rate_once = [False, False, False], # if the fixed_image is not acquired similarly to the moving_image (e.g. RARE to histological (e.g. AMBMC)) this should be False
                          use_histogram_matching = [True, True, True],
                          number_of_iterations = [[3000, 2000], [500, 250], [500, 250]],
                          write_composite_transform = True,
                          collapse_output_transforms = True,
                          winsorize_lower_quantile = 0.005,
                          winsorize_upper_quantile = 0.995,
                          args = '--float',
                          num_threads = 3,
                          initial_moving_transform_com = True,
                         )
reg_struct.run()

RuntimeError: Command:
antsRegistration --float --collapse-output-transforms 1 --dimensionality 3 --initial-moving-transform [ /home/julia/projects/allen_brain/P56_Atlas_different.nii, /home/julia/projects/lc/JH_LC_rsfMRI_03_analysis/struct/struct_masked.nii.gz, 1 ] --initialize-transforms-per-stage 0 --interpolation Linear --output transform --transform Rigid[ 0.1 ] --metric GC[ /home/julia/projects/allen_brain/P56_Atlas_different.nii, /home/julia/projects/lc/JH_LC_rsfMRI_03_analysis/struct/struct_masked.nii.gz, 1, 64, Regular, 0.2 ] --convergence [ 3000x2000, 1e-16, 30 ] --smoothing-sigmas 1.0x0.0vox --shrink-factors 2x1 --use-estimate-learning-rate-once 0 --use-histogram-matching 1 --transform Affine[ 0.1 ] --metric MI[ /home/julia/projects/allen_brain/P56_Atlas_different.nii, /home/julia/projects/lc/JH_LC_rsfMRI_03_analysis/struct/struct_masked.nii.gz, 1, 16, None, 0.3 ] --convergence [ 500x250, 1e-32, 30 ] --smoothing-sigmas 1.0x0.0vox --shrink-factors 1x1 --use-estimate-learning-